In [13]:
import pandas as pd
# import module as mod
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from logging import exception
import joblib as jl



# RECUPERATION DATA

In [160]:
    # load data from csv
    
# data = pd.read_csv('base-joconde-extrait.csv', delimiter=';',low_memory=False ,encoding='utf-8')
# data.dataframeName = 'base-joconde-extrait.csv'

In [161]:
    # copy of data for notload all csv every time
    
# df=data.copy()
# df.shape

In [162]:
    # create  a joblib dump for small sample df
    

# dff=data.sample(frac=0.01)
# jl.dump(dff,'dff.jl')
# dff.shape

In [14]:
    # load the joblib dump 
df = jl.load('dff.jl')
df.shape

(6509, 75)

In [19]:
df = df.reset_index(drop=True)

In [32]:
a=df.loc[65,:]
print(a.Utilisation)

jeu;loisir


## reduction de colonne

In [164]:
# % de remplissage des colonnes
na=df.isnull().sum()/df.shape[0]*100
na=na.sort_values()
na

ID-notice                         0.000000
POP_CONTIENT_GEOLOCALISATION      0.000000
POP_COORDONNEES                   0.000000
Statut juridique                  0.000000
PRODUCTEUR                        0.000000
                                   ...    
MANQUANT_COM                     99.800277
REFMEM                          100.000000
NSDA                            100.000000
REFMER                          100.000000
REFPAL                          100.000000
Length: 75, dtype: float64

In [165]:
# Suppression des colonne supperieur a 50% de NAN
na50=na[na.values<50].index
df=df[na50]

In [166]:
df.columns

Index(['ID-notice', 'POP_CONTIENT_GEOLOCALISATION', 'POP_COORDONNEES',
       'Statut juridique', 'PRODUCTEUR', 'Identifiant Museofile',
       'Numéro de l'objet', 'BASE', 'CONTIENT_IMAGE', 'Lieu de conservation',
       'Domaine', 'Ville_', 'Source et date de la notice', 'NOMOFF', 'DPT',
       'REGION', 'IMAGE', 'Label Musée de France', 'Matériaux-techniques',
       'Dimensions', 'Date d'import', 'LOCA2', 'Période de création', 'Auteur',
       'Date d'acquisition', 'Titre', 'APTN', 'PHOT', 'Dénomination', 'Ecole',
       'Précisions sur l'auteur', 'Date de mise à jour', 'Sujet'],
      dtype='object')

In [167]:
#choix des colonnes utiles
col = ['ID-notice',
    'Auteur',
    'Titre', 
    'Dénomination',
    'Domaine', 
    'Sujet',
    # 'POP_CONTIENT_GEOLOCALISATION', 
    # 'Statut juridique',
    # 'PRODUCTEUR',
    # "Numéro de l'objet",
    'BASE',
    'CONTIENT_IMAGE', 
    # 'Lieu de conservation', 
    'Identifiant Museofile', 
    # 'Source et date de la notice', 
    'NOMOFF',
    'LOCA2', 
    'REGION', 
    'DPT', 
    'Ville_',
    'POP_COORDONNEES',
    # 'IMAGE', 
    # 'Label Musée de France', 
    # 'Matériaux-techniques',
    # 'Dimensions', 
    # "Date d'import", 
    # 'Période de création', 
    # "Date d'acquisition", 
    # 'APTN', 
    # 'PHOT', 
    # "Précisions sur l'auteur", 
    # 'Ecole', 
    # 'Date de mise à jour', 
    # 'MSGCOM'
]
len(col)

15

In [168]:
df=df[col]
df.shape

(6509, 15)

## clean encodage

- Il y a des lignes qui sont mal encodées
- la colonne base ne contient qu'une valeur. Mais elle en contient des mal encodé. ca nous permetra de recuperer les lignes mal encodées

In [169]:
# recup de la valeur base mal encodé ( c'est le string le plus long)

a=list(df.BASE.unique())
a.sort(key = len, reverse=True)
wrong_base = a[0]

# creation de 2 db , dont 1 mal encodé
df_cp1252=df[df.BASE == wrong_base]
df_ok=df[df.BASE == a[1]]


In [170]:
# verification qu' on recupere bien toutes les données
print(df_cp1252.shape + df_ok.shape)
df_cp1252.shape[0] + df_ok.shape[0]

(422, 15, 6087, 15)


6509

In [171]:
df_cp1252.head(2)

,ID-notice,Auteur,Titre,Dénomination,Domaine,Sujet,BASE,CONTIENT_IMAGE,Identifiant Museofile,NOMOFF,LOCA2,REGION,DPT,Ville_,POP_COORDONNEES
557796,05940003872,JAUREGUI Juan de,Alcazar in Apocalyptus (gravure illustrant le ...,NaN,estampe,"scÃ¨ne biblique (Dieu le PÃ¨re, cheval, TÃ©tra...",Collections des musÃ©es de France (Joconde),oui,M0594,musÃ©e Goya,NaN,Occitanie,Tarn,Castres,"0.0,0.0"
420201,06770010655,MANUFACTURE INDETERMINÃ‰E,Chevaux cabrÃ©s,statuette;paire (2),arts décoratifs;céramique,reprÃ©sentation animaliÃ¨re (cheval),Collections des musÃ©es de France (Joconde),oui,M0677,musÃ©e Thomas-Henry,NaN,Normandie,Manche,Cherbourg-en-Cotentin,"0.0,0.0"


In [172]:
def convert(s):
    if s is not None:
        try:
            if isinstance(s,str):
                s = s.encode("cp1252").decode()
            else : 
                s = str(s).encode("cp1252").decode()
                
        except UnicodeError:
            pass
    return s

In [173]:
columns = ['Auteur','Titre', 'Dénomination', 'Domaine', 'Sujet','BASE','NOMOFF', 'LOCA2','REGION', 'DPT', 'Ville_' ]
for column in columns:
    df_cp1252.loc[:, column] = df.loc[:,column].apply(convert)

C:\Users\IA-Alternant\AppData\Local\Temp\ipykernel_12920\594087213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cp1252.loc[:, column] = df.loc[:,column].apply(convert)
C:\Users\IA-Alternant\AppData\Local\Temp\ipykernel_12920\594087213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cp1252.loc[:, column] = df.loc[:,column].apply(convert)
C:\Users\IA-Alternant\AppData\Local\Temp\ipykernel_12920\594087213.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [174]:
# concatenation des 2 df
df1 = pd.concat([df_cp1252, df_ok])
df1.shape

(6509, 15)

## Suppression des lignes

In [175]:
# Ligne en double
df1 = df1.drop_duplicates(subset=['ID-notice'],keep='first')
df1.shape

(6506, 15)

In [176]:
# Oeuvre n'ayant pas de musé 
df1 = df1.dropna(subset=['Identifiant Museofile'])
df1.shape

(6506, 15)

In [177]:
# #suppression des nan Identifiant Museofile ne contiene aune info  de locallisation
# df1 = df1.loc[df1['Identifiant Museofile'].dropna().index]

# print(df1.shape)
# df1.head(3)

In [178]:
# Mettre des NAN
df1.replace('nan',np.nan, inplace=True)
df1.replace('',np.nan, inplace=True)


In [179]:
df1.Auteur.fillna('inconnu', inplace=True)

## Recup auteur

In [180]:

def nb_autor(aut):
    b=1
    if ';' in aut:
        b=len(aut.split(";"))
    return b
        

In [181]:
def normalize(auts):
    clean_auts=[]
    for aut in auts : 
        # if '(' in aut:
        #     aut=aut.split('(')[0]
        if aut:
            clean_auts.append(aut.lower().strip())

    return  clean_auts

In [182]:
from pandas import isna


def net_aut(aut):
    if not isna(aut) :
        if not isinstance(aut, str):
            aut=str(aut)
            
        if ';' in aut:
            aut = aut.split(';')
        else:
            aut=[aut]

        aut=normalize(aut) 
        
    return aut


In [183]:
# nettoyage des auteur : split , lower , et garder juste le nom/prenom

df1.Auteur=df1.Auteur.apply(net_aut)

In [184]:
# df_aut = df1[['Auteur','ID-notice']]
# df_aut=df_aut.set_index('ID-notice')
# df_aut

In [185]:
lst=df1.Auteur[df1.Auteur.notna()].values.tolist()

In [186]:
aut_lst = []
for auts in lst:
    aut_lst.extend(auts)
aut_lst=list(set(aut_lst))

In [187]:
len(aut_lst)

3068

In [188]:
aut_lst

['la marra franc. (graveur)',
 'aubert & cie (imprimeur)',
 'carracci annibale',
 'pencz georg (dessinateur, graveur)',
 'carucci jacopo (attribué à)',
 'waret',
 'deloras henriette',
 'bra théophile françois marcel',
 'drake thomas françois (dessinateur, peintre)',
 'jourdan louis',
 "tavernier de jonquieres (dessinateur, d'après)",
 'picasso pablo (dit)',
 'clouet françois (atelier)',
 'cambiaso luca',
 'chapu henri michel antoine (dessinateur)',
 'derousse blanche',
 'alo (dit)',
 'gautherin jean',
 "léonard de vinci (attribué, d'après)",
 'richomme jules',
 'compagnie centrale des émeris & produits à polir (commanditaire)',
 'caligula (dit)',
 'van loo louis michel',
 'le gouaz yves marie',
 'masereel frans',
 'gasq paul',
 'daubigny charles francois',
 'olivier-pinot (imprimeur, éditeur)',
 'ubac raoul',
 'feret mascre (usine)',
 'lopez y portana vicente',
 'berchem nicolaes pietersz (graveur, inventeur)',
 'spear j w',
 'sirven bernard (imprimeur)',
 'le hay elisabeth sophie (gra

## Recup musée

In [189]:
df2=df1.copy()

In [190]:
df2['Identifiant Museofile'].unique().shape

(332,)

In [191]:
df2 = df2[['Identifiant Museofile','NOMOFF','REGION','DPT','Ville_','POP_COORDONNEES']]
df2.columns = ['id','nom','region','dept','ville','geoloc']

In [192]:
df2

,id,nom,region,dept,ville,geoloc
557796,M0594,musée Goya,Occitanie,Tarn,Castres,"0.0,0.0"
420201,M0677,musée Thomas-Henry,Normandie,Manche,Cherbourg-en-Cotentin,"0.0,0.0"
622453,M0418,MUS - musée d'histoire urbaine et sociale de S...,Ile-de-France,Hauts-de-Seine,Suresnes,"48.868303,2.221944"
546092,M0866,musée départemental des Hautes-Alpes,Provence-Alpes-Côte d'Azur,Hautes-Alpes,Gap,"44.562625,6.085573"
417272,M0012,musée du Château des Rohan,Grand Est,Bas-Rhin,Saverne,"48.741958,7.362787"
...,...,...,...,...,...,...
102830,M7050,muséum d'histoire naturelle,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Nice,"43.701924,7.279572"
436718,M5001,musée d’archéologie nationale (des origines à ...,Ile-de-France,Yvelines,Saint-Germain-en-Laye,"48.897879,2.094936"
578961,M0332,musée des beaux-arts et d'archéologie,Bourgogne-Franche-Comté,Doubs,Besançon,"47.240055,6.022938"
66897,M0172,musée des ursulines musée Lamartine,Bourgogne-Franche-Comté,Saône-et-Loire,Mâcon,"46.30716,4.832892"


In [193]:
df2.drop(df2[df2['id']== '0000'].index,inplace=True)
df2['id'].replace('5027','M5027',inplace=True)
df2['id']=df2['id'].apply(lambda x: x.upper())  # met en majuscule le Mdevant le numero de musé

In [194]:

# df2['Identifiant Museofile']=df2['Identifiant Museofile'].apply(lambda x: 'M'+x if x[0]!='M' else x)

In [195]:
# ordonne ceux qui ont le plus de données en premier et supprime les doublons
df2['nb_nan']=df2.isna().sum(axis=1)
df2=df2.sort_values('nb_nan')
df2=df2.drop_duplicates(subset='id',keep='first')
df2=df2.drop(columns=['nb_nan'])
df2=df2.sort_values('id')

In [196]:
print(df2.shape)
df2['id'].unique().shape



(331, 6)


(331,)

In [197]:
df2

,id,nom,region,dept,ville,geoloc
418774,B9001,Bibliothèque de l’Assemblée Nationale,Ile-de-France,75007,"Paris, Assemblée nationale","48.86215,2.31897"
428417,B9002,Bibliothèque municipale de Grasse,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Grasse,"43.65559,6.91736"
67354,B9003,Bibliothèque municipale de Besançon,Bourgogne-Franche-Comté,Doubs,Besançon,"47.23649,6.0274"
485964,M0003,musée du Pays de Hanau - histoire et vies d’un...,Grand Est,Bas-Rhin,Bouxwiller,"48.824977,7.482915"
633271,M0008,Maison de l'Archéologie des Vosges du Nord,Grand Est,Bas-Rhin,Niederbronn-les-Bains,"48.947444,7.650428"
...,...,...,...,...,...,...
76028,M7011,muséum d'histoire naturelle,Bourgogne-Franche-Comté,Doubs,Besançon,"47.236687,6.022095"
24468,M7013,muséum d'histoire naturelle,Occitanie,Haute-Garonne,Toulouse,"43.594189,1.448945"
53777,M7015,muséum d'histoire naturelle,Normandie,Seine-Maritime,Le Havre,"49.487552,0.108231"
343360,M7050,muséum d'histoire naturelle,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Nice,"43.701924,7.279572"


In [198]:
df2=df2.reset_index(drop=True)
# df2=df2.fillna(['inconnue'])
df2

,id,nom,region,dept,ville,geoloc
0,B9001,Bibliothèque de l’Assemblée Nationale,Ile-de-France,75007,"Paris, Assemblée nationale","48.86215,2.31897"
1,B9002,Bibliothèque municipale de Grasse,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Grasse,"43.65559,6.91736"
2,B9003,Bibliothèque municipale de Besançon,Bourgogne-Franche-Comté,Doubs,Besançon,"47.23649,6.0274"
3,M0003,musée du Pays de Hanau - histoire et vies d’un...,Grand Est,Bas-Rhin,Bouxwiller,"48.824977,7.482915"
4,M0008,Maison de l'Archéologie des Vosges du Nord,Grand Est,Bas-Rhin,Niederbronn-les-Bains,"48.947444,7.650428"
...,...,...,...,...,...,...
326,M7011,muséum d'histoire naturelle,Bourgogne-Franche-Comté,Doubs,Besançon,"47.236687,6.022095"
327,M7013,muséum d'histoire naturelle,Occitanie,Haute-Garonne,Toulouse,"43.594189,1.448945"
328,M7015,muséum d'histoire naturelle,Normandie,Seine-Maritime,Le Havre,"49.487552,0.108231"
329,M7050,muséum d'histoire naturelle,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Nice,"43.701924,7.279572"


In [199]:
df2


,id,nom,region,dept,ville,geoloc
0,B9001,Bibliothèque de l’Assemblée Nationale,Ile-de-France,75007,"Paris, Assemblée nationale","48.86215,2.31897"
1,B9002,Bibliothèque municipale de Grasse,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Grasse,"43.65559,6.91736"
2,B9003,Bibliothèque municipale de Besançon,Bourgogne-Franche-Comté,Doubs,Besançon,"47.23649,6.0274"
3,M0003,musée du Pays de Hanau - histoire et vies d’un...,Grand Est,Bas-Rhin,Bouxwiller,"48.824977,7.482915"
4,M0008,Maison de l'Archéologie des Vosges du Nord,Grand Est,Bas-Rhin,Niederbronn-les-Bains,"48.947444,7.650428"
...,...,...,...,...,...,...
326,M7011,muséum d'histoire naturelle,Bourgogne-Franche-Comté,Doubs,Besançon,"47.236687,6.022095"
327,M7013,muséum d'histoire naturelle,Occitanie,Haute-Garonne,Toulouse,"43.594189,1.448945"
328,M7015,muséum d'histoire naturelle,Normandie,Seine-Maritime,Le Havre,"49.487552,0.108231"
329,M7050,muséum d'histoire naturelle,Provence-Alpes-Côte d'Azur,Alpes-Maritimes,Nice,"43.701924,7.279572"


In [200]:
df.Auteur

18956          BALTHASAR (dit);HAUG Hans (patronyme)
117593                                MASEREEL Frans
475111                                  BRETON Jules
246051                          CALANDRUCCI Giacinto
526223    HUMBERT Jules Eugène;manufacture de Sèvres
                             ...                    
102830              BARLA Jean-Baptiste (attribué à)
436718                                           NaN
578961                                       anonyme
66897                                            NaN
596226                                   Perrot René
Name: Auteur, Length: 6509, dtype: object

In [201]:
# df.Auteur = df.Auteur.apply(lambda x: ['non renseigné'] if pd.isnull(x) else x)

In [202]:
df1[df1.Auteur.isna()]

,ID-notice,Auteur,Titre,Dénomination,Domaine,Sujet,BASE,CONTIENT_IMAGE,Identifiant Museofile,NOMOFF,LOCA2,REGION,DPT,Ville_,POP_COORDONNEES


In [206]:
df1.Auteur.isna().sum()

0

In [ ]:
df3=df.copy()

In [207]:
# df3.Auteur.fillna('non renseigné')
# df3.Auteur.replace(np.nan,['non renseigné'])

In [208]:
# df3[df3.Auteur.isna()]

In [2]:
import pandas as pd
import module as mod
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from module import *
from mod_dao import * 
import joblib as jl
import os
import mysql.connector

from dotenv import load_dotenv
from dotenv import dotenv_values



In [3]:
config = dotenv_values(".env")
connection = mysql.connector.connect(
    user = config['USER'],
    password = config['PASSWORD'],
    host = config['HOST'],
    database = config['DB_NAME'])
cursor = connection.cursor()

In [10]:
mySql_select_art_id = "SELECT id FROM artiste WHERE nom = %s "
aut= 'gay jacques'
try : 
    cursor.execute(mySql_select_art_id, [aut])  # recupere l'id artiste par le nom
    id_aut = cursor.fetchone()[0]
except Exception as e:
    print (e, "error select id from artiste for ", aut)
    
print(id_aut)



13


In [11]:
del id_aut

In [12]:
mySql_select_art_id = "SELECT id FROM artiste WHERE nom = %s "
aut= 'plop'
try : 
    cursor.execute(mySql_select_art_id, [aut])  # recupere l'id artiste par le nom
    id_aut = cursor.fetchone()[0]
except Exception as e:
    print (e, " / error select id from artiste for ", aut)
    
print(id_aut)

'NoneType' object is not subscriptable  / error select id from artiste for  plop


NameError: name 'id_aut' is not defined

In [ ]:

mySql_insert_art_oeuv = 'INSERT IGNORE INTO art_oeuv (oeuvre, artiste) VALUES(%s,%s)'
mySql_select_art_id = "SELECT id FROM artiste WHERE nom = %s "


for i in range(len(df)):
    line=df.iloc[i,:].values.tolist()
    id_oeuvre=line[0]
    auteurs = line[1]


    for aut in auteurs:
        try : 
            cursor.execute(mySql_select_art_id, [aut])  # recupere l'id artiste par le nom
            id_aut = cursor.fetchone()[0]

            try:
                cursor.execute(mySql_insert_art_oeuv,(id_oeuvre,id_aut))
            except Exception as e:
                print( e,
                    "error insert id: ",
                        id_oeuvre,id_aut
                    )
                continue

        except Exception as e:
            print (e, "error select id from artiste for ", aut)
            continue

    # finally:
connection.commit()
connection.close()    


In [7]:
artiste = "l'hoest eugène"
mySql_select_art_id = "SELECT id FROM artiste WHERE nom = %s "
cursor.execute(mySql_select_art_id, [artiste])  # recupere l'id artiste par le nom
id_aut = cursor.fetchone()[0]
id_aut


32

In [35]:
connection

In [1]:
connection.is_connected()

NameError: name 'connection' is not defined

AttributeError: 'CMySQLCursor' object has no attribute 'is_connected'

In [20]:
connection.close()

In [21]:
connection

In [27]:
cursor.close()

False

In [12]:
cursor

In [33]:
config['USER']

'root'

In [2]:
connection, cursor = new_db()
connection, cursor

ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)

In [9]:
connection

NameError: name 'connection' is not defined

In [10]:
connection.close()

NameError: name 'connection' is not defined

In [2]:
config = dotenv_values(".env")

In [12]:
config

OrderedDict([('USER', 'root'),
             ('PASSWORD', 'toto'),
             ('HOST', '127.0.0.1'),
             ('DB_NAME', 'joconde')])

In [3]:
config['PASSWORD']

'Pa$$0rd'

In [5]:
config['USER']

'root'

In [6]:
config['HOST']

'127.0.0.1'

In [7]:
config['DB_NAME']

'joconde'

In [36]:
s = ["hEllO","iNteRneT","pEopLe"]
s = list(map(lambda x: x.lower(), s))

s

['hello', 'internet', 'people']

In [37]:
df[['ID-notice','Titre','Dénomination','Sujet','Domaine','Identifiant Museofile']]

,ID-notice,Titre,Dénomination,Sujet,Domaine,Identifiant Museofile
0,00160018055,Château de Cheverny,NaN,NaN,dessin,M0016
1,08680000588,Voisins,NaN,NaN,estampe,M0867
2,50350022175,"Autoportrait, de profil vers la droite",NaN,autoportrait,dessin,M5031
3,50350202901,"Religieuse priant agenouillée, reprises des ma...",NaN,NaN,dessin,M5031
4,50190041936,Elément décoratif [La Nature],NaN,NaN,dessin,M5019
...,...,...,...,...,...,...
6504,70500002620,Lentin écailleux ; champignon,NaN,représentation scientifique (champignon),sciences naturelles;dessin;mycologie,M7050
6505,5001MIL3645,NaN,armature;flèche,NaN,archéologie;néolithique,M5001
6506,M0332012969,Portrait d'homme,NaN,"portrait d'inconnu (homme, en buste)",dessin,M0332
6507,01720012013,NaN,fuseau,NaN,ethnologie;textile,M0172
